In [2]:
import ccxt.async_support as ccxt
import pandas as pd
import asyncio
import os
import sys
from pprint import pprint

In [3]:
df = pd.read_pickle("binanceus_DOGE-USDT_1620688812541_1620688872541_lob.pkl")

In [7]:
df

,date,exchange_id,symbol,asks,bids
0,2021-05-10T23:20:13.085Z,binanceus,DOGE/USDT,"[[0.4479014, 476.0], [0.4479726, 45.0], [0.448...","[[0.4475, 100.0], [0.4474383, 224.0], [0.44731..."
1,2021-05-10T23:20:13.593Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4479014, 1.0], [0.4479...","[[0.4475796, 292.0], [0.4475, 100.0], [0.44743..."
2,2021-05-10T23:20:14.090Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4478105, 4464.0], [0.4...","[[0.4475796, 292.0], [0.4475, 100.0], [0.44743..."
3,2021-05-10T23:20:14.591Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4478105, 4464.0], [0.4...","[[0.4471995, 4250.0], [0.4470597, 352.0], [0.4..."
4,2021-05-10T23:20:15.166Z,binanceus,DOGE/USDT,"[[0.4476204, 374.0], [0.4477604, 399.0], [0.44...","[[0.4473696, 296.0], [0.4471995, 4250.0], [0.4..."
...,...,...,...,...,...
115,2021-05-10T23:21:10.596Z,binanceus,DOGE/USDT,"[[0.4502588, 518.0], [0.4503693, 1307.0], [0.4...","[[0.4499839, 369.0], [0.4494738, 8235.0], [0.4..."
116,2021-05-10T23:21:11.125Z,binanceus,DOGE/USDT,"[[0.4502588, 518.0], [0.4503693, 1307.0], [0.4...","[[0.4494992, 1084.0], [0.4493381, 8781.0], [0...."
117,2021-05-10T23:21:11.663Z,binanceus,DOGE/USDT,"[[0.4503693, 1307.0], [0.4504798, 1960.0], [0....","[[0.4496608, 868.0], [0.4494992, 1084.0], [0.4..."
118,2021-05-10T23:21:12.090Z,binanceus,DOGE/USDT,"[[0.4503693, 1307.0], [0.4504798, 1960.0], [0....","[[0.4496992, 8133.0], [0.4496608, 868.0], [0.4..."


In [8]:
df['asks'][0]

[[0.4479014, 476.0],
 [0.4479726, 45.0],
 [0.4480104, 408.0],
 [0.4481504, 923.0],
 [0.4481673, 17850.0],
 [0.4482905, 426.0],
 [0.4484005, 1174.0],
 [0.4484295, 17837.0],
 [0.4485105, 1194.0],
 [0.4485374, 44644.0],
 [0.4486205, 1527.0],
 [0.4487305, 166.0],
 [0.4488355, 891.0],
 [0.4490037, 44585.0],
 [0.4490392, 2742.0],
 [0.4492429, 2743.0],
 [0.4494552, 2349.0],
 [0.4496097, 2971.0],
 [0.44976, 3202.0],
 [0.4499031, 2804.0],
 [0.4500462, 3162.0],
 [0.4501894, 3235.0],
 [0.4503682, 3606.0],
 [0.4505913, 3159.0],
 [0.4507473, 7113.0],
 [0.4507474, 3838.0],
 [0.4509034, 3917.0],
 [0.4511484, 2932.0],
 [0.4513934, 3496.0],
 [0.4514073, 24384.0],
 [0.4516218, 4619.0],
 [0.4518271, 2826.0],
 [0.4518503, 3043.0],
 [0.4519916, 3621.0],
 [0.4520678, 78.0],
 [0.4521091, 9160.0],
 [0.4521885, 5775.0],
 [0.4523405, 210.0],
 [0.4523854, 4074.0],
 [0.4525, 100.0]]

In [9]:
df['bids'][0]

[[0.4475, 100.0],
 [0.4474383, 224.0],
 [0.4473188, 304.0],
 [0.4472343, 28.0],
 [0.4470597, 352.0],
 [0.447, 3667.0],
 [0.446888, 401892.0],
 [0.4468873, 46.0],
 [0.446864, 30.0],
 [0.4468421, 17850.0],
 [0.4467617, 105.0],
 [0.4467562, 8111.0],
 [0.4467512, 1952.0],
 [0.4466362, 997.0],
 [0.4465213, 672.0],
 [0.4465212, 3344.0],
 [0.4464812, 44593.0],
 [0.446422, 29.0],
 [0.4463122, 3334.0],
 [0.4462888, 46.0],
 [0.4461159, 48.0],
 [0.4460857, 44631.0],
 [0.4460717, 1917.0],
 [0.4459697, 26.0],
 [0.4458738, 561.0],
 [0.4458312, 3845.0],
 [0.4455961, 45.0],
 [0.4455696, 25.0],
 [0.4455594, 3298.0],
 [0.4454955, 45.0],
 [0.4454462, 37.0],
 [0.4453921, 2825.0],
 [0.4452376, 278.0],
 [0.4452248, 4419.0],
 [0.4452, 23.0],
 [0.4450854, 211.0],
 [0.445, 173.0],
 [0.4448903, 3887.0],
 [0.4448217, 170.0],
 [0.4447022, 3788.0]]